# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,281.15,76,88,2.95,NZ,1690500543
1,1,santa cruz,-17.8000,-63.1667,291.18,88,40,3.09,BO,1690500543
2,2,vorkuta,67.5000,64.0000,287.62,92,97,0.95,RU,1690500543
3,3,nemuro,43.3236,145.5750,290.27,97,75,0.72,JP,1690500543
4,4,cody,44.5263,-109.0565,302.27,30,0,10.29,US,1690500486


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity"

)

# Display the map
map_plot

:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[city_data_df['Humidity'] < 100]

# Drop any rows with null values
city_data_clean = city_data_df.dropna()

# Display sample data
city_data_clean.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,281.15,76,88,2.95,NZ,1690500543
1,1,santa cruz,-17.8000,-63.1667,291.18,88,40,3.09,BO,1690500543
2,2,vorkuta,67.5000,64.0000,287.62,92,97,0.95,RU,1690500543
3,3,nemuro,43.3236,145.5750,290.27,97,75,0.72,JP,1690500543
4,4,cody,44.5263,-109.0565,302.27,30,0,10.29,US,1690500486
6,6,huarmey,-10.0681,-78.1522,294.56,76,16,5.24,PE,1690500544
7,7,aasiaat,68.7098,-52.8699,276.47,88,73,2.34,GL,1690500544
8,8,owase,34.0667,136.2000,301.71,77,34,1.27,JP,1690500544
9,9,yankton,42.8711,-97.3973,316.55,44,0,5.14,US,1690500544
10,10,albany,42.6001,-73.9662,299.38,88,36,0.89,US,1690500545


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    'City': city_data_clean['City'],
    'Country': city_data_clean['Country'],
    'Latitude': city_data_clean['Lat'],
    'Longitude': city_data_clean['Lng'],
    'Humidity': city_data_clean['Humidity']
   
})
hotel_df = city_data_clean[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,papatowai,NZ,-46.5619,169.4708,76,
1,santa cruz,BO,-17.8000,-63.1667,88,
2,vorkuta,RU,67.5000,64.0000,92,
3,nemuro,JP,43.3236,145.5750,97,
4,cody,US,44.5263,-109.0565,30,
...,...,...,...,...,...,...
555,alice town,BS,25.7167,-79.3000,75,
556,jayapura,ID,-2.5333,140.7000,87,
557,karak,JO,31.1667,35.7500,24,
558,irtyshsk,KZ,53.3333,75.4572,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    name_address
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df

Starting hotel search


papatowai - nearest hotel: No hotel found
santa cruz - nearest hotel: Aviador Hotel Boutique
vorkuta - nearest hotel: "Мегаполис"
nemuro - nearest hotel: イーストハーバーホテル
cody - nearest hotel: Comfort Inn At Buffalo Bill Village Resort
huarmey - nearest hotel: Hostal Santa Rosa
aasiaat - nearest hotel: SØMA
owase - nearest hotel: たまつ旅館
yankton - nearest hotel: River Rose Suites
albany - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
suva - nearest hotel: Extension 68
kapuskasing - nearest hotel: Travelodge
wenchang - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
weno - nearest hotel: High Tide Hotel
ikalamavony - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
kodiak - nearest hotel: Shelikof Lodge
sao jose da coroa grande - nearest hotel: Hotel do Sol
coquimbo - 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,papatowai,NZ,-46.5619,169.4708,76,No hotel found
1,santa cruz,BO,-17.8000,-63.1667,88,Aviador Hotel Boutique
2,vorkuta,RU,67.5000,64.0000,92,"""Мегаполис"""
3,nemuro,JP,43.3236,145.5750,97,イーストハーバーホテル
4,cody,US,44.5263,-109.0565,30,Comfort Inn At Buffalo Bill Village Resort
...,...,...,...,...,...,...
555,alice town,BS,25.7167,-79.3000,75,No hotel found
556,jayapura,ID,-2.5333,140.7000,87,Hotel Jasmine Jayapura
557,karak,JO,31.1667,35.7500,24,Al-Mujeb Hotel
558,irtyshsk,KZ,53.3333,75.4572,85,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=['City', 'Hotel Name', 'Country']
)
# Display the map
map_plot

:Points   [Lng,Lat]   (City,Hotel Name,Country)